# Descripción del data set Financial data of 4400+ public companies

✅ ¿Qué es?

Se trata de un conjunto de datos que contiene información financiera de más de 4 400 empresas públicas.  

Los datos fueron raspados (“scraped”) de Yahoo Finance, lo que incluye los estados financieros — el balance general (“balance sheet”), el estado de resultados (“income statement”), y el flujo de caja (“cash flow statement”).  

Incluye datos tanto anuales como trimestrales para un período reciente (aproximadamente los últimos 4 años) para cada empresa.  


# ¿Qué contiene exactamente?

Algunas de las características clave de la base de datos:

- Cada empresa viene identificada con su ticker o símbolo bursátil (o equivalente) y posiblemente con su nombre, industria, etc.
- Para cada empresa, los datos incluyen:
    - Estado de resultados: ingresos, beneficios, gastos, etc.
    - Balance general: activos, pasivos, capital contable, etc.
    - Flujo de caja: flujos operativos, de inversión, de financiación, etc.
- Los datos están organizados para que puedas ver la evolución por trimestre y por año, lo que permite análisis de tendencias.
- Se pueden usar variables financieras como predictors (por ejemplo: activos, pasivos, ingresos) y variables objetivo como beneficio, rentabilidad, crecimiento, etc.

# Paso 1 — Reconocer el dataset

1. Objetivo: investigar el dataset entregado en la asignación 2, revisar el nombre de cada columna y documentar qué representa cada una (tipo, unidad, periodicidad, observaciones).

2. Pasos recomendados:
    - Cargar el archivo entregado (por ejemplo: df = pd.read_csv(...)) y listar columnas: df.columns
    - Para cada columna: buscar su significado (glosario, documentación de la fuente, o inspección de valores).
    - Anotar tipo de dato, unidad (USD, porcentaje, entero), periodicidad (trimestral/anual) y cualquier observación (por ejemplo: calculada, acumulada, neta/bruta).
    - Generar la lista final con nombre y descripción clara y breve.

3. Ejemplo de Plantilla para la lista (reemplazar con las columnas reales del dataset):

- ticker: Identificador bursátil de la empresa (string). Ejemplo: "AAPL".
- fiscal_date: Fecha del periodo financiero (YYYY-MM-DD). Indica el cierre del trimestre/año.
- revenue: Ingresos netos durante el periodo (num, USD). Periodicidad: trimestral/anual.
- gross_profit: Beneficio bruto (num, USD). Definición: ingresos menos costo de ventas.
- operating_income: Resultado operativo (num, USD). Incluye gastos operativos.
- net_income: Beneficio neto después de impuestos (num, USD).


4. Resultado esperado:
    - Un listado documentado con cada columna del dataset y su descripción (puede entregarse como tabla o como lista de pares nombre→descripción).

In [ ]:
import csv
from datetime import datetime

raw_data = []
with open('balanceSheetHistory_annually.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        parsed = {
            'stock': row['stock'],
            'endDate': datetime.strptime(row['endDate'], '%Y-%m-%d').date(),
        }
        for key, value in row.items():
            if key in ('stock', 'endDate'):
                continue
            try:
                parsed[key] = float(value)
            except (TypeError, ValueError):
                parsed[key] = 0.0
        raw_data.append(parsed)

print(f"Registros cargados: {len(raw_data)}")
print(f"Columnas disponibles: {len(raw_data[0])}")
print("Primer registro:", raw_data[0])


# Paso 2 — Seleccionar columnas relevantes

Después de haber cargado el dataset, elimina todas las columnas y quédate solo con: `stock`, `endDate` y `cash`.

- Verifica que los nombres y la capitalización de las columnas sean correctos antes de seleccionar.


In [ ]:
relevant_columns = [
    'stock',
    'endDate',
    'cash',
    'totalAssets',
    'totalLiab',
    'totalCurrentAssets',
    'totalCurrentLiabilities',
]

selected_data = [
    {col: row[col] for col in relevant_columns}
    for row in raw_data
]

print('Columnas usadas:', relevant_columns)
print('Ejemplo de fila:', selected_data[0])


# Paso 3 — Separar por empresas y elegir 3 para predecir su cantidad de dinero por fecha

- Objetivo: crear series temporales por empresa usando las columnas `stock`, `endDate` y `cash` y seleccionar 3 empresas para modelar y predecir `cash` por fecha.
- Pasos recomendados:
    - Verificar que las columnas `stock`, `endDate` y `cash` existen y están limpias (sin valores nulos o con imputación cuando sea necesario).
    - Ordenar el dataframe por `stock` y `endDate` (ascendente) para obtener la serie temporal de cada empresa.
    - Agrupar por `stock` y crear un subset por empresa.
    - Elegir 3 empresas con suficientes observaciones (p. ej. mayor número de fechas disponibles o relevancia del negocio).
    - Para cada empresa seleccionada, preparar los datos de entrenamiento/validación (features temporales, ventanas, lag, etc.) y definir la variable objetivo `cash` por `endDate`.

In [ ]:
from collections import defaultdict

target_stocks = ['IVC', 'CSLT', 'LPX']
company_data = defaultdict(list)

for row in selected_data:
    if row['stock'] in target_stocks:
        company_data[row['stock']].append(row)

for stock in target_stocks:
    company_data[stock].sort(key=lambda r: r['endDate'])

def ascii_plot(series, label, width=40):
    values = [r['cash'] for r in series]
    min_v, max_v = min(values), max(values)
    span = max(max_v - min_v, 1)
    print(f"
Serie de {label} (cash)")
    for r in series:
        norm = int(((r['cash'] - min_v) / span) * width)
        print(f"{r['endDate']}: {'#' * norm} {r['cash']:.0f}")

for stock in target_stocks:
    ascii_plot(company_data[stock], stock)


# Paso 4 — Grafique tiempo vs dinero de las 3 empresas en 3 gráficas diferentes

Objetivo: visualizar la serie temporal de `cash` frente a `endDate` para cada una de las 3 empresas seleccionadas, colocando cada empresa en una gráfica independiente.

Pasos recomendados:
- Asegurarse de tener el DataFrame con las columnas `stock`, `endDate` y `cash` y ordenado por `stock` y `endDate` ascendente.
- Convertir `endDate` a tipo fecha: `df['endDate'] = pd.to_datetime(df['endDate'])`.
- Seleccionar las 3 empresas elegidas: p. ej. `stocks = ['AAA','BBB','CCC']` y crear un subset por cada `stock`.
- Para cada empresa, graficar `endDate` en el eje x y `cash` en el eje y en una figura separada.
- Configurar títulos, etiquetas de ejes y formato de fechas (rotar etiquetas si es necesario). Añadir grid y leyenda si procede.
- Opcional: usar subplots (3 filas x 1 columna) para mostrar las 3 gráficas en la misma figura o generar 3 figuras individuales según preferencia.
- Guardar las figuras si es necesario: `plt.savefig('cash_stock_AAA.png', bbox_inches='tight')`.

Ejemplo de librerías a usar: matplotlib, seaborn o plotly para interactividad.

# Paso 5 — División 80/20 (entrenamiento / prueba)

Objetivo: separar los datos en 80% para entrenamiento y 20% para prueba respetando la estructura temporal por empresa (sin hacer shuffle).

Recomendaciones:
- Asegúrate de tener las columnas `stock`, `endDate` y `cash` y que el dataframe esté ordenado por `stock` y `endDate` (ascendente).
- Para series temporales por empresa, usa una división basada en tiempo: los primeros 80% de observaciones de cada `stock` → train; los últimos 20% → test.



In [ ]:
def split_train_test(series, train_ratio=0.8):
    cutoff = max(1, int(len(series) * train_ratio))
    train = series[:cutoff]
    test = series[cutoff:]
    return train, test

splits = {}
for stock in target_stocks:
    train, test = split_train_test(company_data[stock])
    splits[stock] = {'train': train, 'test': test}
    print(f"{stock}: entrenamiento={len(train)} | prueba={len(test)}")


# Paso 6 — crea y entrena el modelo de regresión lineal para predecir `cash` por empresa

Objetivo: entrenar un modelo de regresión lineal por cada `stock` usando la serie temporal (respetando orden temporal) y evaluar en el 20% final.

Pasos recomendados:
- Preprocesamiento
    - Definir variables para el entrenamiento: y = `cash` (variable objetivo). `data` → X (conjunto de características a usar para predecir `cash`, p. ej. rezagos de `cash`, indicadores temporales, variables exógenas). Usar X e y en el entrenamiento: `model.fit(X_train, y_train)`.
    - Asegurar `endDate` como datetime y ordenar por `stock`, `endDate`.  - Asegurar `endDate` como datetime y ordenar por `stock`, `endDate`.



In [ ]:
from typing import List


def transpose(matrix: List[List[float]]):
    return [list(row) for row in zip(*matrix)]


def matmul(a: List[List[float]], b: List[List[float]]):
    result = [[0.0 for _ in range(len(b[0]))] for _ in range(len(a))]
    for i in range(len(a)):
        for k in range(len(b)):
            for j in range(len(b[0])):
                result[i][j] += a[i][k] * b[k][j]
    return result


def invert(matrix: List[List[float]]):
    n = len(matrix)
    aug = [row[:] + [1.0 if i == j else 0.0 for j in range(n)] for i, row in enumerate(matrix)]

    for col in range(n):
        pivot_row = max(range(col, n), key=lambda r: abs(aug[r][col]))
        aug[col], aug[pivot_row] = aug[pivot_row], aug[col]
        pivot = aug[col][col]
        if pivot == 0:
            raise ValueError('Matrix is singular and cannot be inverted')
        aug[col] = [val / pivot for val in aug[col]]
        for r in range(n):
            if r == col:
                continue
            factor = aug[r][col]
            aug[r] = [rv - factor * cv for rv, cv in zip(aug[r], aug[col])]

    return [row[n:] for row in aug]


def train_linear_regression(X: List[List[float]], y: List[float]):
    Xt = transpose(X)
    XtX = matmul(Xt, X)
    XtX_inv = invert(XtX)
    XtY = matmul(Xt, [[val] for val in y])
    weights = matmul(XtX_inv, XtY)
    return [w[0] for w in weights]


def build_features(rows: List[dict]):
    features = []
    targets = []
    for r in rows:
        features.append([
            1.0,
            r['totalAssets'],
            r['totalLiab'],
            r['totalCurrentAssets'],
        ])
        targets.append(r['cash'])
    return features, targets


models = {}
feature_names = ['bias', 'totalAssets', 'totalLiab', 'totalCurrentAssets']

for stock, split in splits.items():
    X_train, y_train = build_features(split['train'])
    weights = train_linear_regression(X_train, y_train)
    models[stock] = weights
    print(f"{stock}: " + ', '.join(f"{name}={weight:.4f}" for name, weight in zip(feature_names, weights)))


# Paso 7 — Verifica tu modelo de regresión lineal: grafica real vs predicho para las 3 empresas

- Objetivo: comparar visualmente los valores reales de `cash` del conjunto de prueba con los valores predichos por el modelo para cada una de las 3 empresas seleccionadas.
- Requisitos: tener `endDate` como datetime, el conjunto test por cada `stock`, y las predicciones (`y_pred`) para cada test.
- Pasos recomendados:
    - Para cada empresa (stock):
        - Extraer test: filas finales (20%) ordenadas por `endDate`.
        - Obtener predicciones usando el modelo entrenado: `y_pred = model.predict(X_test)`.
        - Crear una gráfica con `endDate` en el eje x y ambos: `cash` real (línea/points) y `cash` predicho (línea punteada) en el eje y.
        - Añadir título con el ticker, leyenda, etiquetas de ejes y grid. Formatear fechas y rotar etiquetas si hace falta.
    - Opcional: mostrar las 3 series en subplots (3 filas x 1 columna) para facilitar comparación.
    - Calcular y mostrar métricas de error por empresa (MAE, RMSE, R2) bajo cada gráfico o en una tabla resumen.
- Resultado esperado: tres gráficas (una por empresa) mostrando real vs predicho y una tabla o texto con las métricas de evaluación.

In [ ]:
def predict(weights, features):
    return sum(w * x for w, x in zip(weights, features))


def prepare_features(rows):
    return [[1.0, r['totalAssets'], r['totalLiab'], r['totalCurrentAssets']] for r in rows]

for stock, split in splits.items():
    X_test = prepare_features(split['test'])
    y_test = [r['cash'] for r in split['test']]
    preds = [predict(models[stock], feats) for feats in X_test]
    split['preds'] = preds
    split['y_test'] = y_test

    print(f"
{stock} — valores reales vs predichos")
    for row, y_hat in zip(split['test'], preds):
        print(f"{row['endDate']}: real={row['cash']:.2f} | predicho={y_hat:.2f}")


# Paso 8 — Verificación de modelos (MSE, RMSE, R2)

Instrucciones breves:
- Asegúrese de tener para cada empresa: y_test (valores reales) y y_pred (predicciones).
- Calcular métricas con sklearn: mean_squared_error(y_test, y_pred), RMSE = sqrt(MSE), r2_score(y_test, y_pred).
- Presentar los resultados en una tabla por empresa y añadir una conclusión corta.

Empresa 1 (ticker: AAA)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Empresa 2 (ticker: BBB)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Empresa 3 (ticker: CCC)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Pequeña conclusión:
- Comparar RMSE/MSE absolutos para evaluar error en unidades de `cash`; RMSE más bajo = mejor precisión.
- R2 indica proporción de varianza explicada (cercano a 1 → buen ajuste; cercano a 0 o negativo → mal ajuste).
- Si alguno de los modelos muestra RMSE alto o R2 bajo, considerar: más features (rezagos, variables temporales), regularización, transformación de la serie o modelos no lineales.

In [ ]:
import math


def mse(y_true, y_pred):
    return sum((a - b) ** 2 for a, b in zip(y_true, y_pred)) / len(y_true)


def rmse(y_true, y_pred):
    return math.sqrt(mse(y_true, y_pred))


def r2_score(y_true, y_pred):
    mean_y = sum(y_true) / len(y_true)
    ss_total = sum((val - mean_y) ** 2 for val in y_true)
    ss_res = sum((a - b) ** 2 for a, b in zip(y_true, y_pred))
    return 1 - ss_res / ss_total if ss_total != 0 else 0.0


for stock, split in splits.items():
    if not split['y_test']:
        print(f"{stock}: sin datos suficientes para métricas")
        continue

    y_true = split['y_test']
    y_pred = split['preds']
    print(
        f"
{stock} -> MSE: {mse(y_true, y_pred):.2f}, "
        f"RMSE: {rmse(y_true, y_pred):.2f}, R2: {r2_score(y_true, y_pred):.4f}"
    )


## Preguntas de Analisis 

1. ¿Qué variables (features) usaste para predecir `cash` y por qué crees que son relevantes?
2. ¿Cómo realizaste la división temporal 80/20 por empresa y por qué es importante no barajar (shuffle) los datos en series temporales?
3. ¿Cuál es la diferencia entre MSE y RMSE y qué nos dice cada métrica sobre la precisión de las predicciones?
4. ¿Qué interpreta el valor de R² en este problema (predicción de `cash`) y qué limitaciones tiene su interpretación en series temporales?
5. Si obtienes un RMSE alto o un R² bajo, ¿qué acciones concretas propondrías para mejorar el modelo (p. ej. features, rezagos, transformaciones, modelos alternativos)?